In [133]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import datetime
import os.path

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [134]:
model_name = 'faceDetectModel'
savedModelPath = '../log/faceDetectModel'
tb_log_dir = "../log/tensorboard/faceDetectModel"
cp_filepath = '../log/cps/faceDetectModel/latest_weights.h5'

batch_size = 32
epochs = 10

## Model callbacks

In [135]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
                    # Checkpoint callback                    
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=False),

                    # Tensorboard callback
                    tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
            ]

## Data parameters

In [136]:
image_height = 224
image_width = 224

## Creating the training dataset

In [137]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 84 files belonging to 2 classes.
Using 68 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [138]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

Found 84 files belonging to 2 classes.
Using 16 files for validation.


## Load the base model

In [139]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50(
                include_top=False,
                weights='imagenet',  # Load weights pre-trained on ImageNet.
                input_shape=(image_height, image_width, 3)
                )
base_model.trainable = False

## Creating a new model

In [140]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

# Loading latest saved weights into model
if(os.path.isfile(cp_filepath)):
    print('Loading weights...')
    model.load_weights(filepath=cp_filepath)

## Training the new model with new data

In [141]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 0.4484 - accuracy: 0.8088
Epoch 1: saving model to ../log/cps/faceDetectModel\latest_weights.h5
3/3 [==============================] - 35s 8s/step - loss: 0.4484 - accuracy: 0.8088 - val_loss: 0.2374 - val_accuracy: 0.9375
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 0.2490 - accuracy: 0.9559
Epoch 2: saving model to ../log/cps/faceDetectModel\latest_weights.h5
3/3 [==============================] - 14s 5s/step - loss: 0.2490 - accuracy: 0.9559 - val_loss: 0.1479 - val_accuracy: 0.9375
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 0.1487 - accuracy: 0.9853
Epoch 3: saving model to ../log/cps/faceDetectModel\latest_weights.h5
3/3 [==============================] - 14s 5s/step - loss: 0.1487 - accuracy: 0.9853 - val_loss: 0.0832 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 0.0781 - accuracy: 1.0000
Epoch 4: saving model to ../log/cps/faceD

## Save the model

In [142]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../log/faceDetectModel\assets


INFO:tensorflow:Assets written to: ../log/faceDetectModel\assets


## Predict new images

In [143]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))

1/1 [==============================] - 3s 3s/step
This image contains a face. (96.47 percent confidence)
